In [2]:
# Packages
import pandas as pd
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

from bokeh.io import output_notebook#, show
from bokeh.models import DataRange1d
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
output_notebook()

def plot_config(width: int, height: int,title=None,
                x_label: str = None, 
                y_label: str = None):
    p = figure(title=title, background_fill_color="#fafafa", width=width, height=height)
    p.output_backend = "svg"    # save plot in HTML as SVG
    if title!=None:
        p.title.text_font_size = "12pt"
    p.xaxis.axis_label = x_label
    p.yaxis.axis_label = y_label
    p.xaxis.ticker.num_minor_ticks=0
    p.yaxis.ticker.num_minor_ticks=0
    p.yaxis.axis_label_text_font_size = "12pt"
    p.yaxis.major_label_text_font_size = "12pt"
    p.xaxis.major_label_text_font_size = "12pt"
    p.xaxis.axis_label_text_font_size = "12pt"
    return p

Loading BokehJS ...

# Load Sim Result

In [ ]:
# file_name = 'KF0'
# file_name = 'KF_noise'
file_name = 'MHE0'
# file_name = 'MHE_noise'

# name = 'CD-EKF'
name = 'MHE'

df_sim = pd.read_csv('./sim_result/' + file_name + '.csv')
columns = df_sim.columns.to_list()
t0 = df_sim['time'][0]

cond1 = df_sim['time'] - t0 >= 10
cond2 = df_sim['time'] - t0 <= 20
df_sim = df_sim[cond1*cond2]

gpr_t0 = 10
gpr_tn = 20

df_sim.drop('Unnamed: 0', axis=1, inplace=True)

df_tf = df_sim[df_sim['sensor.type'] == 'tf']


In [ ]:
print('min: ',min(deta_tt)*1000)
print('max: ',max(deta_tt)*1000)
print('mean: ',np.mean(deta_tt)*1000)
print('median: ',np.median(deta_tt)*1000)



# Euclidean Error based on GPR of TF Data

In [ ]:
ti = np.array(df_tf['time'] - t0).reshape(-1,1)
xi = np.array(df_tf['tf.x']).reshape(-1)
yi = np.array(df_tf['tf.y']).reshape(-1)

kernel = 3 * RBF(length_scale=0.7, length_scale_bounds='fixed')
gpr_x = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0)
gpr_x.fit(ti, xi)
gpr_y = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0)
gpr_y.fit(ti, yi)

condi1 = df_sim['time'] - t0 >= gpr_t0
condi2 = df_sim['time'] - t0 <= gpr_tn
df_window = df_sim[condi1*condi2]

t_gpr = df_window['time'].to_numpy().reshape(-1,1) - t0
x_gpr = gpr_x.predict(t_gpr)
y_gpr = gpr_y.predict(t_gpr)
t_gpr = t_gpr.reshape(-1)

df_win_rt = df_window[-df_window['rt_x'].isna()]

t_rt_gpr = df_win_rt['time'].to_numpy().reshape(-1,1) - t0
x_rt_gpr = gpr_x.predict(t_rt_gpr)
y_rt_gpr = gpr_y.predict(t_rt_gpr)
t_rt_gpr = t_rt_gpr.reshape(-1)

# Mean of Absolute Error (MAE) with GPR
dx_rt = df_win_rt['rt_x'].to_numpy() - x_rt_gpr
dy_rt = df_win_rt['rt_y'].to_numpy() - y_rt_gpr
dx = df_window['x'].to_numpy() - x_gpr
dy = df_window['y'].to_numpy() - y_gpr
norm2 = np.sqrt(dx ** 2 + dy ** 2)
norm2_rt = np.sqrt(dx_rt ** 2 + dy_rt ** 2)
mae_offline = np.average(norm2)
mae_rt = np.average(norm2_rt)

In [ ]:
cir_size = 2
p_total = []

if name=='CD-EKF':
    color = 'royalblue'
elif name=='MHE':
    color = 'orange'

# Trajectory
p1 = plot_config(width=400, height=300, x_label='x [ m ]', y_label='y [ m ] ')#, title='Trajectory')
p1.circle(x=x_rt_gpr, y=y_rt_gpr, legend_label='GPR', size=cir_size, fill_color='MediumSeaGreen',line_color='MediumSeaGreen')
p1.circle(x=df_sim['rt_x'], y=df_sim['rt_y'], legend_label=name, size=cir_size, fill_color=color, line_color=color)
p1.circle(x=df_tf['tf.x'], y=df_tf['tf.y'], legend_label='CAM data', size=cir_size, fill_color='black',line_color='black')
p1.circle(x=df_sim['rt_x'].iloc[0], y=df_sim['rt_y'].iloc[0], size=10, fill_color='crimson', line_color='crimson')
p1.square(x=df_sim['rt_x'].iloc[-1], y=df_sim['rt_y'].iloc[-1], size=10, fill_color='crimson', line_color='crimson')
p1.text(df_sim['rt_x'].iloc[0], df_sim['rt_y'].iloc[0],['Start'])
p1.text(df_sim['rt_x'].iloc[-1], df_sim['rt_y'].iloc[-1],['End'])
# p1.legend.nrows=1
p1.legend.ncols=1
# p1.legend.location = "bottom_left"
p1.match_aspect=True
p1.aspect_scale=1
p_total.append(p1)

p2 = plot_config(width=400, height=300, x_label='x [ m ]', y_label='y [ m ] ')#, title='Trajectory zoom in')
p2.circle(x=x_rt_gpr, y=y_rt_gpr, legend_label='GPR', size=cir_size, fill_color='MediumSeaGreen',line_color='MediumSeaGreen')
p2.circle(x=df_sim['rt_x'], y=df_sim['rt_y'], legend_label=name, size=cir_size, fill_color=color, line_color=color)
p2.circle(x=df_tf['tf.x'], y=df_tf['tf.y'], legend_label='CAM data', size=cir_size, fill_color='black',line_color='black')
p2.y_range = DataRange1d(start=-2.3, end=-1.9)
p2.x_range = DataRange1d(start=3.55, end=4)
p2.xaxis.ticker.min_interval=0.1
p2.yaxis.ticker.min_interval=0.1
p2.match_aspect=True
p2.aspect_scale=1
p2.legend.location = "bottom_left"
p_total.append(p2)

# # Euclidean Error
t_ee = t_rt_gpr-t_rt_gpr[0]
p20 = plot_config(width=400, height=200, x_label='t [ s ]', y_label=r'Euclidean Error [ m ]')
p20.circle(x=t_ee, y=norm2_rt, legend_label='between '+name+' and GPR', size=cir_size, fill_color=color, line_color=color)
p20.line(x=[t_ee[0], t_ee[-1]], y=[mae_rt]*2, legend_label=f'MAE=avg={mae_rt:.4f}, std={np.std(norm2_rt):.4f}', line_width=2, line_dash=[6,3], line_color='black')
p20.legend.ncols=1
p20.legend.location = "top_left"
p_total.append(p20)


# # Computational Time
t_exemax = max(df_sim['t_com'])*1000
p4 = plot_config(width=400, height=200, x_label='t [ s ]', y_label='Execution Time [ ms ] ')
p4.circle(x=df_sim['time'] - t0- 10, y=df_sim['t_com']*1000, legend_label='t_exe, '+name, size=cir_size, fill_color=color, line_color=color)

if 't_nlp' in columns:
    p4.circle(x=df_sim['time'] - t0 -10, y=df_sim['t_nlp']*1000, legend_label='t_nlp', size=cir_size, fill_color='slateblue', line_color='slateblue')
p4.line(x=[df_sim['time'].iloc[0] - t0 -10, df_sim['time'].iloc[0-1] - t0 -10], 
        y=[t_exemax]*2,
        legend_label = f'max = {t_exemax:.2f} ms', line_width=2, line_dash=[2,1], line_color='red')
# p4.legend.location = "bottom_left"
p4.legend.location = "top_left"
# p4.legend.nrows=2
p_total.append(p4)

for i in p_total:
    show(i, notebook_handle=True)

# Load both EKF MHE

In [3]:
# file_ekf = 'KF0'
# file_mhe = 'MHE0'
# file_ekf = 'KF_noise'
# file_mhe = 'MHE_noise'
# file_ekf = 'KF_delay200'
# file_mhe = 'MHE_delay200'
file_ekf = 'KF_sm'
file_mhe = 'MHE_sm'

df_ekf = pd.read_csv('./sim_result/' + file_ekf + '.csv')
df_mhe = pd.read_csv('./sim_result/' + file_mhe + '.csv')

t0 = df_ekf['time'][0]

cond1 = df_ekf['time'] - t0 >= 10
cond2 = df_ekf['time'] - t0 <= 20

df_ekf = df_ekf[cond1*cond2]
df_mhe = df_mhe[cond1*cond2]

gpr_t0 = 10
gpr_tn = 20

df_ekf.drop('Unnamed: 0', axis=1, inplace=True)
df_mhe.drop('Unnamed: 0', axis=1, inplace=True)

df_tf = df_ekf[df_ekf['sensor.type'] == 'tf']


# Euclidean Error based on GPR of TF Data

In [4]:
ti = np.array(df_tf['time'] - t0).reshape(-1,1)
xi = np.array(df_tf['tf.x']).reshape(-1)
yi = np.array(df_tf['tf.y']).reshape(-1)

kernel = 3 * RBF(length_scale=0.7, length_scale_bounds='fixed')
gpr_x = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0)
gpr_x.fit(ti, xi)
gpr_y = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0)
gpr_y.fit(ti, yi)

condi1 = df_ekf['time'] - t0 >= gpr_t0
condi2 = df_ekf['time'] - t0 <= gpr_tn
df_win_ekf = df_ekf[condi1*condi2]
df_win_mhe = df_mhe[condi1*condi2]

t_gpr = df_ekf['time'].to_numpy().reshape(-1,1) - t0 # use ekf, ekf mhe are same here
x_gpr = gpr_x.predict(t_gpr)
y_gpr = gpr_y.predict(t_gpr)
t_gpr = t_gpr.reshape(-1)

# time steps of real time result of 2 methods are different
df_win_ekf_rt = df_win_ekf[-df_win_ekf['rt_x'].isna()]
df_win_mhe_rt = df_win_mhe[-df_win_mhe['rt_x'].isna()]

t_rt_gpr_ekf = df_win_ekf_rt['time'].to_numpy().reshape(-1,1) - t0
t_rt_gpr_mhe = df_win_mhe_rt['time'].to_numpy().reshape(-1,1) - t0
x_rt_gpr_ekf = gpr_x.predict(t_rt_gpr_ekf)
x_rt_gpr_mhe = gpr_x.predict(t_rt_gpr_mhe)
y_rt_gpr_ekf = gpr_y.predict(t_rt_gpr_ekf)
y_rt_gpr_mhe = gpr_y.predict(t_rt_gpr_mhe)
t_rt_gpr_ekf = t_rt_gpr_ekf.reshape(-1)
t_rt_gpr_mhe = t_rt_gpr_mhe.reshape(-1)

# Mean of Absolute Error (MAE) with GPR
def mae(df_win_rt, x_rt_gpr, y_rt_gpr):
    dx_rt = df_win_rt['rt_x'].to_numpy() - x_rt_gpr
    dy_rt = df_win_rt['rt_y'].to_numpy() - y_rt_gpr
    norm2_rt = np.sqrt(dx_rt ** 2 + dy_rt ** 2)
    mae_rt = np.average(norm2_rt)
    return norm2_rt, mae_rt
norm2_rt_ekf, mae_rt_ekf = mae(df_win_ekf_rt, x_rt_gpr_ekf, y_rt_gpr_ekf)
norm2_rt_mhe, mae_rt_mhe = mae(df_win_mhe_rt, x_rt_gpr_mhe, y_rt_gpr_mhe)

In [5]:
cir_size = 2
p_total = []

# Trajectory
p1 = plot_config(width=400, height=300, x_label='x [ m ]', y_label='y [ m ] ')#, title='Trajectory')
p1.circle(x=x_gpr, y=y_gpr, legend_label='GPR', size=cir_size,line_color='MediumSeaGreen', fill_color='MediumSeaGreen')
p1.circle(x=df_ekf['rt_x'], y=df_ekf['rt_y'], legend_label='CD-EKF', size=cir_size, line_color='royalblue', fill_color='royalblue')
p1.circle(x=df_mhe['rt_x'], y=df_mhe['rt_y'], legend_label='MHE', size=cir_size, line_color='orange', fill_color='orange')
p1.circle(x=df_ekf['tf.x'], y=df_ekf['tf.y'], legend_label='CAM data', size=cir_size,line_color='black', fill_color='black')

p1.circle(x=df_ekf['rt_x'].iloc[0], y=df_ekf['rt_y'].iloc[0], size=10, fill_color='crimson', line_color='crimson')
p1.square(x=df_ekf['rt_x'].iloc[-1], y=df_ekf['rt_y'].iloc[-1], size=10, fill_color='crimson', line_color='crimson')
p1.text(df_ekf['rt_x'].iloc[0], df_ekf['rt_y'].iloc[0],['Start'])
p1.text(df_ekf['rt_x'].iloc[-1], df_ekf['rt_y'].iloc[-1],['End'])
# p1.legend.nrows=1
p1.legend.ncols=1
p1.legend.location = "bottom_left"
p1.match_aspect=True
p1.aspect_scale=1
p_total.append(p1)

p2 = plot_config(width=400, height=300, x_label='x [ m ]', y_label='y [ m ] ')#, title='Trajectory zoom in')
p2.circle(x=x_gpr, y=y_gpr, legend_label='GPR', size=cir_size, fill_color='MediumSeaGreen',line_color='MediumSeaGreen')
p2.circle(x=df_ekf['rt_x'], y=df_ekf['rt_y'], legend_label='CD-EKF', size=cir_size, fill_color='royalblue', line_color='royalblue')
p2.circle(x=df_mhe['rt_x'], y=df_mhe['rt_y'], legend_label='MHE', size=cir_size, fill_color='orange', line_color='orange')
p2.circle(x=df_tf['tf.x'], y=df_tf['tf.y'], legend_label='CAM data', size=cir_size, fill_color='black',line_color='black')
p2.y_range = DataRange1d(start=-2.3, end=-1.9)
p2.x_range = DataRange1d(start=3.55, end=4)
p2.xaxis.ticker.min_interval=0.1
p2.yaxis.ticker.min_interval=0.1
p2.match_aspect=True
p2.aspect_scale=1
p2.legend.location = "bottom_left"
p_total.append(p2)


p2 = plot_config(width=400, height=250, x_label='x [ m ]', y_label='y [ m ] ')#, title='Trajectory zoom in')
p2.circle(x=x_gpr, y=y_gpr, legend_label='GPR', size=cir_size, fill_color='MediumSeaGreen',line_color='MediumSeaGreen')
p2.circle(x=df_ekf['rt_x'], y=df_ekf['rt_y'], legend_label=f'CD-EKF, MAE = {mae_rt_ekf*1000:.1f}mm', size=cir_size, fill_color='royalblue', line_color='royalblue')
p2.circle(x=df_mhe['rt_x'], y=df_mhe['rt_y'], legend_label=f'MHE, MAE = {mae_rt_mhe*1000:.1f}mm', size=cir_size, fill_color='orange', line_color='orange')
# p2.circle(x=df_tf['tf.x'], y=df_tf['tf.y'], legend_label='CAM data + N~(0, 0.01)', size=cir_size, fill_color='black',line_color='black')
p2.circle(x=df_tf['tf.x'], y=df_tf['tf.y'], legend_label='CAM original data', size=cir_size, fill_color='black',line_color='black')

# p2.y_range = DataRange1d(start=-2.3, end=-1.9)
# p2.x_range = DataRange1d(start=3.55, end=4)
p2.y_range = DataRange1d(start=-2.3, end=-1.85) # larger
p2.x_range = DataRange1d(start=3.35, end=4.0)

p2.xaxis.ticker.min_interval=0.1
p2.yaxis.ticker.min_interval=0.1
p2.match_aspect=True
p2.aspect_scale=1
p2.legend.location = "bottom_left"
p_total.append(p2)

# # Euclidean Error
t_ee_ekf = t_rt_gpr_ekf - t_rt_gpr_ekf[0]
t_ee_mhe = t_rt_gpr_mhe - t_rt_gpr_mhe[0]
p20 = plot_config(width=400, height=300, x_label='t [ s ]', y_label=r'Euclidean Error [ m ]')
p20.circle(x=t_ee_ekf, y=norm2_rt_ekf, legend_label='CD-EKF', size=cir_size, fill_color='royalblue', line_color='royalblue')
p20.line(x=[t_ee_ekf[0], t_ee_ekf[-1]], y=[mae_rt_ekf]*2, legend_label=f'MAE = {mae_rt_ekf*1000:.1f}mm, std = {np.std(norm2_rt_ekf)*1000:.1f}mm', line_width=2, line_dash=[6,3], line_color='blue')
p20.circle(x=t_ee_mhe, y=norm2_rt_mhe, legend_label='MHE', size=cir_size, fill_color='orange', line_color='orange')
p20.line(x=[t_ee_mhe[0], t_ee_mhe[-1]], y=[mae_rt_mhe]*2, legend_label=f'MAE = {mae_rt_mhe*1000:.1f}mm, std = {np.std(norm2_rt_mhe)*1000:.1f}mm', line_width=2, line_dash=[6,3], line_color='darkorange')

p20.legend.ncols=1
p20.legend.location = "top_left"
p_total.append(p20)

for i in p_total:
    show(i, notebook_handle=True)
    


In [ ]:
df_cut = df_ekf[2967:3080]
p = plot_config(width=400, height=200, x_label='x [ m ]', y_label='y [ m ] ')#, title='Trajectory zoom in')
p.circle(x=df_cut['time'] - t0, y=df_cut['rt_x'], legend_label='GPR', size=cir_size, fill_color='MediumSeaGreen',line_color='MediumSeaGreen')

show(p, notebook_handle=True)

print(df_cut['rt_x'][6561])

print(df_cut['time'][6561] - t0)
print(df_cut['time'][6533] - t0)
print('dt=',(df_cut['time'][6561]-df_cut['time'][6533])*1000,'ms')